# Fully local webscraper for manufacturing

### ENPM692 Final Project
Vinay Lanka | Apoorv Thapliyal | Harsh Senjaliya

## Imports

In [1]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Langchain setup
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_ollama import OllamaLLM
import json, re, textwrap, time
from typing import Dict, List, Tuple

#DDG
from duckduckgo_search import DDGS

# Fetching HTML
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer
from typing import List
from langchain.schema import BaseRetriever, Document

# Web scraping
import asyncio

# Database
from bs4 import BeautifulSoup
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms.base import BaseLLM
from langchain.prompts import PromptTemplate

import chromadb
from chromadb.config import Settings
from langchain_community.embeddings import HuggingFaceEmbeddings
from chromadb.config import Settings, DEFAULT_TENANT, DEFAULT_DATABASE
from langchain_community.vectorstores import Chroma

import os, glob
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.vectorstores import Chroma, FAISS
import shutil
from chromadb.config import Settings

from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
import base64, pathlib

/home/vinaylanka/miniconda3/envs/ragenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


## Captioning


In [2]:
# Hugging Face Image Captioning Model Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Print device information
print(f"Using device: {device}")

Using device: cuda


### Using Gemma to generate captions

In [ ]:
img_b64 = base64.b64encode(pathlib.Path("imgs/pencil.jpeg").read_bytes()).decode()

content = [
    {  # image part
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{img_b64}",
    },
    {  # text prompt part
        "type": "text",
        "text": "Identify the primary object in this image and enumerate all observable material characteristics—such as base material, surface finish, color, texture, gloss level, or coating.  Ignore background elements and give the answer in one clear English sentence.",
    },
]

llm = ChatOllama(model="gemma3:4b", temperature=0.3)
object_desc = llm.invoke([HumanMessage(content=content)]).content

print(object_desc)

The primary object in the image is a red plastic water bottle with a gray lid and handle, exhibiting a glossy, smooth surface finish with a white graphic design printed on its exterior.


## Static DB

### Setup static db 

Chroma DB with Persistence
Embedding LLM - all-MiniLM-L12-v2

Converts documents under `/data` to vector embeddings under `/db`.
Place any documents to be stored under `/data`.
To be run once.

In [4]:
static_dir = "db/sdb"
embedder   = HuggingFaceEmbeddings(model_name="all-MiniLM-L12-v2")

client = chromadb.PersistentClient(
    path=static_dir,
    settings=Settings(),          # you can pass custom Settings here if needed
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

if os.path.exists(static_dir) and os.listdir(static_dir):
    print(f"Loading existing vector store from {static_dir}")
    vectordb_static = Chroma(
        client=client,
        embedding_function=embedder,
        persist_directory=static_dir
    )

else:
    print(f"Building new vector store in {static_dir}")
    if os.path.exists(static_dir):
        shutil.rmtree(static_dir)
    os.makedirs(static_dir, exist_ok=True)

    # load PDFs
    pdf_docs = []
    for path in glob.glob("data/*.pdf"):
        loader = PyPDFLoader(path)
        pdf_docs.extend(loader.load())

    # split into chunks
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=200,
        length_function=len
    )
    static_chunks = splitter.split_documents(pdf_docs)

    # build & persist
    vectordb_static = Chroma.from_documents(
        documents=static_chunks,
        embedding=embedder,
        client=client,
        persist_directory=static_dir
    )
    client.persist() 

/tmp/ipykernel_116617/2846763773.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder   = HuggingFaceEmbeddings(model_name="all-MiniLM-L12-v2")


Loading existing vector store from db/sdb


/tmp/ipykernel_116617/2846763773.py:13: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb_static = Chroma(


### Create a retreiver

Pulling k = 6 relevant chunks

In [5]:
static_ret  = vectordb_static.as_retriever(
    search_kwargs={"k": 6}
)

## Generate queries

The LLM Model: Ollama gemma3:4b model will be used for query generation as well as final manufacturing document generation

In [6]:
# Initialize your LLM
llm = OllamaLLM(model="gemma3:4b", temperature=0.3)

query_prompt = PromptTemplate.from_template(textwrap.dedent("""
You are a manufacturing‐research assistant.

Goal ▸ Draft **{k}** DuckDuckGo search queries that, taken together,
will surface authoritative information for *all* of the report areas
listed under **SECTIONS** below.

Guidelines
• Draw wording **only** from the context slide excerpts, the object
  caption, and the section descriptions.  
• Mix high-level and specific terms so the result set spans every
  section (sustainability, process flow, tooling, simulation, etc.).  
• Avoid near-duplicate phrasing; each query should probe a different
  angle (process, material, KPI, cost, digital thread, …).  
• Do **not** introduce themes absent from the context.  
• Return **only** a valid JSON array of strings—no commentary.

---
CONTEXT
{context}
---

OBJECT DESCRIPTION
\"\"\"{caption}\"\"\"

SECTIONS (for your internal guidance; do NOT echo them)
1  Executive summary / Object overview
2  Sustainability & life-cycle considerations
3  Material selection & eco-alternatives
4  Manufacturing-process flow
5  Tooling, automation & industrial robotics
6  Digital & smart-manufacturing enablers
7  Simulation & virtual validation
8  Quality & performance metrics
9  Environmental & cost impact
10 Implementation roadmap / Smart-mfg priorities
---
"""))


def generate_queries(caption: str,
                     static_ret,
                     k_queries: int = 6,
                     k_ctx: int = 4) -> list[str]:


    docs = static_ret.get_relevant_documents(caption)[:k_ctx]
    ctx  = "\n\n".join(d.page_content[:800] for d in docs)


    chain  = LLMChain(llm=llm,
                      prompt=query_prompt.partial(k=k_queries))

    result_dict = chain.invoke({"caption": caption, "context": ctx})
    raw = result_dict["text"] if isinstance(result_dict, dict) else result_dict

    match = re.search(r"\[.*\]", raw, re.DOTALL)
    if not match:
        raise ValueError(f"JSON not found:\n{raw}")

    return [q.strip() for q in json.loads(match.group(0))]


In [7]:
search_queries = generate_queries(object_desc, static_ret, k_queries=10, k_ctx=6)
print("Generated queries:")
for q in search_queries:
    print(f"  • {q}")

/tmp/ipykernel_116617/3696445281.py:50: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = static_ret.get_relevant_documents(caption)[:k_ctx]
/tmp/ipykernel_116617/3696445281.py:54: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain  = LLMChain(llm=llm,


Generated queries:
  • red plastic water bottle material composition
  • water bottle manufacturing process flow diagram
  • glossy plastic surface finish properties
  • water bottle tooling automation cost analysis
  • digital thread integration water bottle production
  • simulation virtual validation water bottle design
  • water bottle life cycle assessment sustainability
  • KPIs for plastic water bottle manufacturing quality
  • water bottle environmental impact carbon footprint
  • smart manufacturing priorities plastic bottle production


## Dynamic DB

### Duck-Duck-Go scraping for top N URLs

In [8]:
def ddg_search(query: str, max_results: int = 5) -> list[str]:
    """
    Returns the top‑N DuckDuckGo result URLs for a query.
    """
    with DDGS() as ddgs:
        return [hit["href"] for hit in ddgs.text(query, max_results=max_results)]

# Example usage:
# urls = [u for q in queries for u in ddg_search(q, max_results=5)]
# urls = list(dict.fromkeys(urls))  # dedupe while preserving order

In [9]:
async def fetch_documents(urls: list[str]):
    """
    Asynchronously downloads each page and converts HTML→plain text.
    Returns a list of LangChain Document objects.
    """
    loader    = AsyncHtmlLoader(urls)
    html_docs = await loader.load()                       # list[Document] with HTML in .page_content
    transformer = Html2TextTransformer()                  # strips tags, yields markdown‑style text :contentReference[oaicite:0]{index=0}
    text_docs = transformer.transform_documents(html_docs)
    return text_docs


In [10]:
all_urls = []
for q in search_queries:
    urls = ddg_search(q, max_results=5)
    print(f"\nQuery: {q}\n  URLs:")
    for u in urls:
        print("    •", u)
    all_urls.extend(urls)
    time.sleep(1)  # be nice to DDG

unique_urls = list(dict.fromkeys(all_urls))
print(f"\nTotal unique URLs: {len(unique_urls)}")



Query: red plastic water bottle material composition
  URLs:
    • https://www.sciencing.com/raw-materials-plastic-bottles-5747796/
    • https://www.chemistryislife.com/the-chemistry-of-plastic-bottles-pet-hdpe-ldpe
    • https://www.epa.gov/facts-and-figures-about-materials-waste-and-recycling/plastics-material-specific-data
    • https://healthywaterbottle.com/maintenance-health/what-are-plastic-water-bottles-made-of/
    • https://www.myownwater.com/blog/what-is-a-water-bottle-made-of

Query: water bottle manufacturing process flow diagram
  URLs:
    • https://ibottling.com/what-is-bottled-water-production-process-flow-chart/
    • https://www.hzmmachine.com/blog/bottled-water-production-process-pdf.html
    • https://dtppl.com/how-does-packed-drinking-water-manufacturing-facility-work/
    • https://blog.fhyzics.net/sop/sop-manual-for-bottled-water-manufacturing
    • https://www.sartorius.com/download/1085710/beverage-qc-water-workflow-poster-en-print-l-sartorius-pdf-data.pdf



### Fetch HTML Docs as plain text

In [11]:
N = len(unique_urls)
print(f"\nFetching and parsing first {N} available pages:")

html_docs = []
success_count = 0
failure_count = 0

for idx, url in enumerate(unique_urls[:N], start=1):
    single_loader = AsyncHtmlLoader([url])
    try:
        docs_for_url = await asyncio.wait_for(
            asyncio.to_thread(single_loader.load),
            timeout=10
        )
        html_docs.extend(docs_for_url)
        print(f"[{idx}/{N}] ✓ Fetched {url}")
        success_count += 1

    except asyncio.TimeoutError:
        failure_count += 1
        print(f"[{idx}/{N}] ✗ Timeout {url}")
    except Exception as e:
        failure_count += 1
        print(f"[{idx}/{N}] ✗ Error {url}: {e}")

print(f"\nCompleted: {success_count} succeeded, {failure_count} failed out of {N} URLs.\n")

# Transform and inspect as before
transformer = Html2TextTransformer()
docs = transformer.transform_documents(html_docs)

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len
)
ddb = splitter.split_documents(docs)


Fetching and parsing first 49 available pages:


Fetching pages: 100%|##########| 1/1 [00:00<00:00, 16.34it/s]


[1/49] ✓ Fetched https://www.sciencing.com/raw-materials-plastic-bottles-5747796/


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  1.07it/s]


[2/49] ✓ Fetched https://www.chemistryislife.com/the-chemistry-of-plastic-bottles-pet-hdpe-ldpe


...

[47/49] ✓ Fetched https://happyeconews.com/the-future-of-plastic-manufacturing-sustainability-and-innovation/


Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.70s/it]


[48/49] ✓ Fetched https://hannahgthompson.com/innovations-in-plastic-bottle-manufacture-advancements-driving-sustainability-and-efficiency/


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  1.37it/s]


[49/49] ✓ Fetched https://rcbottles.com/innovative-manufacturing-processes-for-plastic-bottles-balancing-efficiency-and-sustainability/

Completed: 48 succeeded, 1 failed out of 49 URLs.



### Convert fetched HTML into LangChain Documents

In [12]:
embedder = SentenceTransformerEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)

vectordb_dynamic = FAISS.from_documents(
    documents=ddb,    # your list of Document chunks
    embedding=embedder
)

## Create Retrievers

In [13]:
class CombinedRetriever(BaseRetriever):
    retrievers: List[BaseRetriever]
    k: int = 5

    def get_relevant_documents(self, query: str) -> List[Document]:
        docs: List[Document] = []
        for r in self.retrievers:
            docs.extend(r.get_relevant_documents(query))
        return docs[: self.k]


dynamic_ret = vectordb_dynamic.as_retriever(
    search_kwargs={"k": 6}
)

combined_retriever = CombinedRetriever(
    retrievers=[static_ret, dynamic_ret],
    k=12
)


/tmp/ipykernel_116617/663454979.py:1: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class CombinedRetriever(BaseRetriever):


## Ask the LLM

Finally, prompt the LLM with the final query

In [ ]:
first_prompt = PromptTemplate.from_template("""
You are a senior manufacturing engineer.

Use the **context** block (course‐slide excerpts + web snippets/URLs) as your only source.
Let the lecture slides define the structure, emphasis, and terminology—shape all web‐sourced
details to fit those slide‐based frameworks.  In other words, pull facts from URLs **only** to
illustrate slide‐prescribed methods, metrics, or priorities, and never stray outside the slide
guidance.

┌───────────────────────────────────────────────────────────────┐
│  Slides commonly cited                                        │
│  • Lecture 1  – course intro & final-project guidance         │
│  • Lecture 4  – sustainable-manufacturing overview, 6 R’s     │
│  • Lecture 5  – unit-process KPIs & material I/O tables       │
└───────────────────────────────────────────────────────────────┘


**Write these three sections (≈ 500 words each), each starting with exactly `## <section title>`:**
- Executive summary / Object overview 
    • Provide a concise description of the object, its primary purpose, and scope.  
    • Summarize the high-level manufacturing objectives and key expected outcomes.  
    • Highlight any unique design features or performance targets. 
- Sustainability & life-cycle considerations  
   • Frame the product within a triple-bottom-line perspective (environmental, social, economic).  
   • Outline the life-cycle stages (raw material sourcing, production, use, end-of-life) with a placeholder LCA flow diagram.  
   • List key sustainability KPIs (e.g., carbon footprint per unit, energy consumption, recyclability rate).
- Material selection & eco-alternatives  
   • Identify primary material properties required (strength, durability, chemical resistance).  
   • Compare candidate materials in a table: embodied energy, cost, recyclability, material toxicity.  
   • Propose eco-alternative materials or blends (bio-based polymers, recycled feedstocks) and substitution strategies.

Formatting rules  
• Start every section with `## <section title>` (no numbers, no bold).  
• Write in formal, technical style with bullets under each heading.  
• Cite every factual claim: `(Lecture #)` or `(URL n)`.  
• Never invent content or sources.

---
CONTEXT
{context}
---

Question → "{question}"

Answer:
""")

second_prompt = PromptTemplate.from_template("""
You are a senior manufacturing engineer.

Use the **context** block (course‐slide excerpts + web snippets/URLs) as your only source.
Let the lecture slides define the structure, emphasis, and terminology—shape all web‐sourced
details to fit those slide‐based frameworks.  In other words, pull facts from URLs **only** to
illustrate slide‐prescribed methods, metrics, or priorities, and never stray outside the slide
guidance.

┌───────────────────────────────────────────────────────────────┐
│  Key slide sources for these sections                         │
│  • Lectures 2-3 – additive & traditional process chains        │
│  • Lecture 5  – unit-process stage/KPI charts                  │
│  • Lecture 7  – industrial-robot anatomy, automation levels    │
│  • Lecture 1  – automation-level pyramid (overview)            │
│  • Lecture 9  – digital-manufacturing information models       │
│  • Lecture 10 – network-centric / smart-manufacturing decks    │
└───────────────────────────────────────────────────────────────┘

**Write the following sections — target ≈ 500 words EACH — with headings:**

- Manufacturing-process flow  
    • Provide a layer-by-layer or unit-operation flow chart description.  
    • Include placeholders for cycle-time/takt-time where slides give cues.

- Tooling, automation & industrial robotics  
    • Bill-of-tooling and robot-cell layout derived from Lecture 7 diagrams.  
    • Map each tool/robot to the step in Section 4; outline an automation-migration strategy.

- Digital & smart-manufacturing enablers  
    • Summarise digital-thread architecture and data-interoperability standards  
        (ISA-95, RAMI 4.0) referenced in Lectures 9 & 10.  
    • List IoT sensors or MES data points that close the loop.

Formatting rules  
• Begin every section with `## <section title>` (Markdown H-2).  
• Write in a formal, technical style, with bulleted points where appropriate.
• Cite every factual claim: `(Lecture #)` for slides, `(URL n)` for
  web snippets where *n* is the URL’s position in the context list.  
• Never invent content or sources.

---
CONTEXT
{context}
---

Question → "{question}"

Answer:
""")

third_prompt = PromptTemplate.from_template("""
You are a senior manufacturing engineer.

Use the **context** block (course‐slide excerpts + web snippets/URLs) as your only source.
Let the lecture slides define the structure, emphasis, and terminology—shape all web‐sourced
details to fit those slide‐based frameworks.  In other words, pull facts from URLs **only** to
illustrate slide‐prescribed methods, metrics, or priorities, and never stray outside the slide
guidance.

┌────────────────────────────────────────────────────────────────────────┐
│  Slide sets most relevant to these sections                           │
│  • Lecture 8  – simulation in manufacturing                            │
│  • Lecture 5  – KPI hierarchy & unit-process metrics                   │
│  • Lecture 4  – life-cycle costing & environmental impact              │
│  • Lecture 10 – smart-manufacturing “ten priority actions”             │
└────────────────────────────────────────────────────────────────────────┘

**Write the following sections — target ≈ 500 words EACH — with headings:**

- Simulation & virtual validation  
    • Explain the rationale for discrete-event simulation.  
    • Sketch a SIM-model swim-lane or text diagram.  
    • List KPIs to test and add a bullet list of risks/benefits.

- Quality & performance metrics  
    • Build a three-tier KPI table (process, product, sustainability) based on Lecture 5.  
    • Mention gauge R&R or similar validation where cited.

- Environmental & cost impact  
    • Provide an energy/CO₂-per-unit chart description and a cost-break-down structure.  
    • Summarise key life-cycle-cost drivers and note “win-win” trade-offs.

- Implementation roadmap / Smart-mfg priorities  
    • Draft a phased Gantt-style narrative with migration milestones.  
    • Align each phase to the “ten priority actions” slide (Lecture 10).  
    • Include a brief stakeholder-responsibility matrix.

Formatting rules  
• Begin every section with `## <section title>` (Markdown H-2).  
• Write in a formal, technical style, with bulleted points where appropriate.
• Cite every factual claim: `(Lecture #)` for slides, `(URL n)` for
  web snippets where *n* is the URL’s position in the context list.  
• Never invent content or sources.

---
CONTEXT
{context}
---

Question → "{question}"

Answer:
""")

section_batches: List[Tuple[str, PromptTemplate, str]] = [
    (
        "Sections 1–3",
        first_prompt,
        "\n".join([
            "1. **Executive summary / Object overview**",
            "2. **Sustainability & life-cycle considerations**",
            "3. **Material selection & eco-alternatives**",
            "",
            f"Object description: {object_desc}"
        ])
    ),
    (
        "Sections 4–6",
        second_prompt,
        "\n".join([
            "4. **Manufacturing-process flow**",
            "5. **Tooling, automation & industrial robotics**",
            "6. **Digital & smart-manufacturing enablers**",
            "",
            f"Object description: {object_desc}"
        ])
    ),
    (
        "Sections 7–10",
        third_prompt,
        "\n".join([
            "7. **Simulation & virtual validation**",
            "8. **Quality & performance metrics**",
            "9. **Environmental & cost impact**",
            "10. **Implementation roadmap / Smart-mfg priorities**",
            "",
            f"Object description: {object_desc}"
        ])
    ),
]

def run_batch(
    prompt_tpl: PromptTemplate,
    question: str
) -> str:
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=combined_retriever,
        chain_type="stuff",
        chain_type_kwargs={
            "prompt": prompt_tpl,
            "document_variable_name": "context",
        },
        input_key="question",
        return_source_documents=False,
    )
    out = qa({"question": question})
    return out["result"] if isinstance(out, dict) else out

def clean_to_first_h2(md: str) -> str:
    idx = md.find("## ")
    return md[idx:].strip() if idx != -1 else md.strip()

report_sections: Dict[str,str] = {}
for batch_name, prompt_tpl, question in section_batches:
    print(f"Generating {batch_name}…")
    raw = run_batch(prompt_tpl, question)
    report_sections[batch_name] = clean_to_first_h2(raw)

with open("manufacturing_report.md", "w") as md:
    for key in ["Sections 1–3", "Sections 4–6", "Sections 7–10"]:
        md.write(report_sections[key] + "\n\n")

print("✅ Done writing manufacturing_report.md")

Generating Sections 1–3…
Generating Sections 4–6…
Generating Sections 7–10…
✅ Done writing manufacturing_report.md


### Executive Overview & Sustainability Foundations

In [113]:
first_prompt = PromptTemplate.from_template("""
You are a senior manufacturing engineer.

Use ONLY the information in the **context** block (course-slide excerpts
and web snippets/URLs) to draft the first three sections of a manufacturing
report.  Let the slides guide the emphasis; enrich with web details when the
context includes them.

┌───────────────────────────────────────────────────────────────┐
│  Slides commonly cited                                        │
│  • Lecture 1  – course intro & final-project guidance         │
│  • Lecture 4  – sustainable-manufacturing overview, 6 R’s     │
│  • Lecture 5  – unit-process KPIs & material I/O tables       │
└───────────────────────────────────────────────────────────────┘

**Write these three sections — ~500 words EACH — and label them exactly:**

1. Executive summary / Object overview  
2. Sustainability & life-cycle considerations  
3. Material selection & eco-alternatives  

Formatting rules  
• Start each section with `## <section title>` (Markdown H-2).  
• Inline cite every factual claim: `(Lecture #)` for slides, `(URL n)` for
  web snippets where *n* is the URL’s position in the context list.  
• If a necessary fact is missing, omit it — never invent content or sources.

---
CONTEXT
{context}
---

Question → "{question}"

Answer:
""")

question = (
    "Task: Draft a detailed section 1,2,3 sections of a manufacturing guide (≈ 1000 words)\n\n"
    "1. **Executive summary / Object overview**\n\n"
    "2. **Sustainability & life-cycle considerations**\n\n"
    "3. **Material selection & eco-alternatives**\n\n"
    "Ground rules:\n"
    "1. The **Object description** below is the object being manufactured.\n"
    "2. All other statements MUST be supported by the CONTEXT block.\n"
    "3. Do not fabricate data or references. \n\n"
    f"Object description: {object_desc}"
)


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=combined_retriever,
    chain_type="stuff",
    chain_type_kwargs={
        "prompt": first_prompt,
        "document_variable_name": "context",
    },
    input_key="question",          # Match the prompt
    return_source_documents=True,
)

result = qa_chain({"question": question})

In [115]:
print(result["result"])

Okay, here's a draft of the three sections you requested, based on the provided context. I've aimed for approximately 1000 words total, prioritizing detail and accuracy based on the provided sources.  I've included citations at the end for easy reference. **Please read the notes at the very end of this response – they are crucial for understanding the limitations and assumptions made in this draft.**

---

**1. Executive Summary / Object Overview: The Rubik's Cube – A Manufacturing Perspective**

The Rubik's Cube, initially known as the Magic Cube, is a globally recognized mechanical puzzle invented by Hungarian architect Ernő Rubik in 1974. It quickly transcended its origins as a teaching tool, becoming a cultural icon and a symbol of problem-solving and ingenuity. This manufacturing guide details the production process of a standard 3x3x3 Rubik’s Cube, focusing on material selection, assembly, and emerging sustainability considerations.  The core challenge in Rubik's Cube manufacturi

### Process Flow, Automation & Digital Enablers

In [78]:
second_prompt = PromptTemplate.from_template("""
You are a senior manufacturing engineer.

Using ONLY the **context** block (slide excerpts and web snippets/URLs),
draft **Sections 4-6** of the manufacturing report.  Prioritise the listed
lectures when choosing evidence; add web details only if they appear in the
context.

┌───────────────────────────────────────────────────────────────┐
│  Key slide sources for these sections                         │
│  • Lectures 2-3 – additive & traditional process chains        │
│  • Lecture 5  – unit-process stage/KPI charts                  │
│  • Lecture 7  – industrial-robot anatomy, automation levels    │
│  • Lecture 1  – automation-level pyramid (overview)            │
│  • Lecture 9  – digital-manufacturing information models       │
│  • Lecture 10 – network-centric / smart-manufacturing decks    │
└───────────────────────────────────────────────────────────────┘

**Write the following sections — target ≈ 500 words EACH — with headings:**

4. Manufacturing-process flow  
   • Provide a layer-by-layer or unit-operation flow chart description.  
   • Include placeholders for cycle-time/takt-time where slides give cues.

5. Tooling, automation & industrial robotics  
   • Bill-of-tooling and robot-cell layout derived from Lecture 7 diagrams.  
   • Map each tool/robot to the step in Section 4; outline an automation-migration strategy.

6. Digital & smart-manufacturing enablers  
   • Summarise digital-thread architecture and data-interoperability standards  
     (ISA-95, RAMI 4.0) referenced in Lectures 9 & 10.  
   • List IoT sensors or MES data points that close the loop.

Formatting rules  
• Begin every section with `## <section title>` (Markdown H-2).  
• Inline-cite every factual claim: `(Lecture #)` for slides, `(URL n)` for
  web snippets where *n* is the order of the URL in the context list.  
• If the context lacks specific details, leave them blank or mark “TBD”
  — never invent numbers, standards, or sources.

---
CONTEXT
{context}
---

Question → "{question}"

Answer:
""")

question = (
    "Task: Draft a detailed section 4,5,6 of a manufacturing guide (≈ 1000 words)\n\n"
    "4. **Manufacturing-process flow**  \n\n"
    "5. **Tooling, automation & industrial robotics**  \n\n"
    "6. **Digital & smart-manufacturing enablers**  \n\n"
    "Ground rules:\n"
    "1. The **Object description** below is the object being manufactured.\n"
    "2. All other statements MUST be supported by the CONTEXT block.\n"
    "3. Do not fabricate data or references. \n\n"
    f"Object description: {object_desc}"
)


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=combined_retriever,
    chain_type="stuff",
    chain_type_kwargs={
        "prompt": second_prompt,
        "document_variable_name": "context",
    },
    input_key="question",          #  <<< match the prompt
    return_source_documents=True,
)

result = qa_chain({"question": question})


In [79]:
print(result["result"])

## Section 4: Manufacturing Process Flow – Rubik’s Cube Production

The production of a Rubik’s Cube is a surprisingly complex process, involving multiple stages of molding, assembly, and finishing. This section details the key steps involved, drawing directly from the provided context.

**4.1 Molding the Core & Center Cubes:**

The foundation of the Rubik’s Cube is the nylon core, followed by the ABS center cubes. The process begins with the injection molding of the nylon core. The context highlights the importance of material shrinkage, stating, “Different plastics will have a different shrink rate, and each tool must be specifically designed for the material that will be used.” This emphasizes the precision required to ensure consistent cube dimensions. The molding process utilizes ABS (Acrylonitrile Butadiene Styrene) for the center cubes, again acknowledging the need for tailored tooling based on material properties. The process involves injecting molten ABS into two-piece molds, 

### Simulation, Metrics & Smart-Manufacturing Roadmap

In [80]:
third_prompt = PromptTemplate.from_template("""
You are a senior manufacturing engineer.

Rely exclusively on the **context** block (course-slide excerpts + web snippets/URLs)
to draft **Sections 7-10** of the manufacturing report.  Give priority to the lecture
slides noted below; supplement with web data only when present in context.

┌────────────────────────────────────────────────────────────────────────┐
│  Slide sets most relevant to these sections                           │
│  • Lecture 8  – simulation in manufacturing                            │
│  • Lecture 5  – KPI hierarchy & unit-process metrics                   │
│  • Lecture 4  – life-cycle costing & environmental impact              │
│  • Lecture 10 – smart-manufacturing “ten priority actions”             │
└────────────────────────────────────────────────────────────────────────┘

**Write the following sections – aim for ≈ 400–500 words EACH – and label them
exactly as shown.  Start each with `## <section title>` (Markdown H-2).**

7. Simulation & virtual validation  
   • Explain the rationale for discrete-event simulation.  
   • Sketch a SIM-model swim-lane or text diagram.  
   • List KPIs to test and add a bullet list of risks/benefits.

8. Quality & performance metrics  
   • Build a three-tier KPI table (process, product, sustainability) based on Lecture 5.  
   • Mention gauge R&R or similar validation where cited.

9. Environmental & cost impact  
   • Provide an energy/CO₂-per-unit chart description and a cost-break-down structure.  
   • Summarise key life-cycle-cost drivers and note “win-win” trade-offs.

10. Implementation roadmap / Smart-mfg priorities  
    • Draft a phased Gantt-style narrative with migration milestones.  
    • Align each phase to the “ten priority actions” slide (Lecture 10).  
    • Include a brief stakeholder-responsibility matrix.

Citation rules  
• After every factual statement, cite its source: `(Lecture #)` or `(URL n)`  
  where *n* is the URL’s order in CONTEXT.  
• If a detail is missing, mark “TBD” — do **not** invent data or sources.

---
CONTEXT
{context}
---

Question → "{question}"

Answer:
""")

question = (
    "Task: Draft a detailed section 7,8,9,10 of a manufacturing guide (≈ 1000 words)\n\n"
    "7. **Simulation & virtual validation**   \n\n"
    "8. **Quality & performance metrics**  \n\n"
    "9. **Environmental & cost impact**  \n\n"
    "10. **Implementation roadmap / Smart-mfg priorities**  \n\n"
    "Ground rules:\n"
    "1. The **Object description** below is the object being manufactured.\n"
    "2. All other statements MUST be supported by the CONTEXT block.\n"
    "3. Do not fabricate data or references. \n\n"
    f"Object description: {object_desc}"
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=combined_retriever,
    chain_type="stuff",
    chain_type_kwargs={
        "prompt": third_prompt,
        "document_variable_name": "context",
    },
    input_key="question",          #  <<< match the prompt
    return_source_documents=True,
)

result = qa_chain({"question": question})


In [81]:
print(result["result"])

Okay, here’s a draft of sections 7-10 of a manufacturing guide for Rubik's Cubes, aiming for approximately 1000 words and incorporating the provided context.

---

**Manufacturing Guide: Rubik’s Cube Production**

**Section 7: Simulation & Virtual Validation (Approx. 250 words)**

Before commencing physical production, a rigorous simulation and virtual validation phase is crucial. This phase leverages CAD (Computer-Aided Design) software and finite element analysis (FEA) to identify potential weaknesses and optimize the cube’s structural integrity. The simulation process begins with a detailed 3D model of each component – the core, the side pieces, and the stickers – generated from the provided context.  Specifically, the simulation will focus on stress distribution under rotational forces, mimicking the cube's intended use. 

The simulation will test various rotational speeds and forces, mirroring the expected user experience.  The context highlights the use of Nylon for the core, ABS